<a href="https://colab.research.google.com/github/urieliram/Pyomo_example/blob/main/UCP_Pyomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q pyomo

\begin{align*}
\text{Min} \quad & \sum_{t\in T} \sum_{j\in J} ( c_{j} p_{jt} + c_{j}^\text{U} y_{jt}) \\
\text{Sujeto a:} \quad & \sum_{j\in J} p_{jt} = D_{t}, & t \in T \\
& \sum_{j\in J} \bar{p}_{jt} \geq D_{t} + R_{t}, &  t \in T\\
& c_{j}p_{jt} \geq \alpha_{js} p_{jt} + \beta_{js}, & s=1,…C_j, j \in J \\
& v_{j,t-1} - v_{jt} + y_{jt} - z_{jt} = 0 & j\in J, t \in T  \\
& p_{jt} - p_{j,t-1} \leq R_{j}^\text{U} v_{j,t-1} + S_{j}^\text{U} y_{jt} & j\in J, t \in T  \\
& p_{j,t-1} - p_{jt} \leq R_{j}^\text{D} v_{j,t} + S_{j}^\text{D} z_{jt} & j\in J, t \in T  \\
& \sum_{k=t-T_{j}^{U}+1,k \geq 1}^{t} y_{jk} \leq v_{jt} & j\in J, t\in [L_{j}+1,...,|T|] \\
& v_{jt}+\sum_{k=t-T_{j}^{D}+1,k \geq 1}^{t} z_{jk} \leq 1 & j\in J, t\in [F_{j}+1,...,|T|] \\
& P_{j}^{\text{min}} v_{jt} \leq p_{jt} \leq \bar{p}_{jt} \leq P_{j}^{\text{max}} v_{jt} & j\in J, t \in T  \\
& \bar{p}_{jt} \leq p_{j,t-1} + R_{j}^{\text{U}} v_{j,t-1} + S_{j}^{\text{U}} Y_{jt} & j\in J, t \in T  \\
& \bar{p}_{jt} \leq P_{j}^{\text{max}} (v_{jt}-z_{j,t+1})+ z_{j,t+1} S_{j}^{\text{D}} & j\in J, t \in T  \\
& P_{jt}, \bar{jt} \geq 0 \\
& v_{jt}, y_{jt}, z_{jt} = \{0,1\}
\end{align*}

Unit Commitment in Electric Energy Systems. M. F. Anjos and A. J. Conejo.  Foundations and Trends in Electric Energy Systems. Vol. 1, No. 4, 220--310 , 2017

In [21]:
T = {1,2,3,4,5,6}
G = {1,2,3}
Pmax= {300,200,100}
Pmin= {80,50,30}
cU = {800,500,250}
c  = {5,15,30}
De = {240,250,200,170,230,190}
R  = {0,10,10,10,10,10}
RD = {30,40,50}
RU = {50,60,70}
SU = {100,70,40}
SD = {80,50,30}
TU = {3,2,1}
TD = {2,2,2}
U  = {2,0,0}
D  = {0,0,0}
v0 = {1,0,0}
p0 = {120,0,0}

In [47]:
from pyomo.environ import *
# Crear un modelo
model      = ConcreteModel()
# Conjuntos
model.G    = Set(initialize = G)
model.T    = Set(initialize = T)
# Parametros
model.Pmax  = Param(model.G    , initialize = Pmax  , within = Any)
model.Pmin  = Param(model.G    , initialize = Pmin  , within = Any)
model.c     = Param(model.G    , initialize = c     , within = Any)
model.cU    = Param(model.G    , initialize = cU    , within = Any)
model.De    = Param(model.T    , initialize = De    , within = Any)
model.R     = Param(model.T    , initialize = R     , within = Any)
model.RU    = Param(model.G    , initialize = RU    , within = Any)
model.RD    = Param(model.G    , initialize = RD    , within = Any)
model.SU    = Param(model.G    , initialize = SU    , within = Any)
model.SD    = Param(model.G    , initialize = SD    , within = Any)
model.U     = Param(model.G    , initialize = U     , within = Any)
model.D     = Param(model.G    , initialize = D     , within = Any)
model.TU    = Param(model.G    , initialize = TU    , within = Any)
model.TD    = Param(model.G    , initialize = TD    , within = Any)
model.v0    = Param(model.G    , initialize = v0   , within = Any)
model.p0    = Param(model.G    , initialize = p0   , within = Any)

# Variables
model.p     = Var( model.G , model.T , bounds = (0.0,99999.0))
model.pb    = Var( model.G , model.T , bounds = (0.0,99999.0))
model.u     = Var( model.G , model.T , within = Binary)
model.v     = Var( model.G , model.T , within = Binary)
model.w     = Var( model.G , model.T , within = Binary)

# Objective Function
def obj_rule(m):
    return  m.total_cEN
model.obj = Objective(rule = obj_rule, sense=minimize)

def total_cEN_rule(m):  ## to account energy cost
    return m.total_cEN == sum( (m.c[j] * m.p[j,t] + m.cU[j] * m.y[j,t]) for t in m.T for j in m.J)
model.total_cEN_ = Constraint(rule = total_cEN_rule)


# def demand_rule65(m,t):                      ## demand eq.(65)
#     ##return sum( m.p[g,t] for g in m.G ) +  m.sn[t]  == m.De[t]
#     return sum( m.p[g,t] for g in m.G )   == m.De[t]
# model.demand_rule65  = Constraint(model.T, rule = demand_rule65)

# def demand_rule67(m,t):                      ## demand + reserve eq.(67)
#     ##return sum( m.pb[g,t] for g in m.G ) +  m.sn[t] >= m.De[t] + m.R[t]
#     return sum( m.pb[g,t] for g in m.G )  >= m.De[t] + m.R[t]
# model.demand_rule67  = Constraint(model.T, rule = demand_rule67)

ERROR:pyomo.core:Rule failed when generating expression for Objective obj with index None:
AttributeError: 'ConcreteModel' object has no attribute 'total_cEN'
ERROR:pyomo.core:Constructing component 'obj' from data=None failed:
AttributeError: 'ConcreteModel' object has no attribute 'total_cEN'


AttributeError: 'ConcreteModel' object has no attribute 'total_cEN'